# Machine Learning Project

##### Imports

In [18]:
%pylab inline
import scipy.io as sio
from sklearn.neighbors import NearestNeighbors
from sklearn import svm, grid_search
from sklearn.cross_validation import cross_val_predict, cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from glob import glob

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


##### Define some variables

In [27]:
def get_variables(data):
    gX, gY, rX, rY = data["goalx"][-120:], data["goaly"][-120:], data["respx"][-120:], data["respy"][-120:]
    tmpY = gX + gY
    mapper = {val: i for i, val in enumerate(sorted(list(set(tmpY[:,0]))))}
    #print "mapping is", mapper
    y = [mapper[Y] for Y in tmpY[:,0]] 
    X = np.hstack((rX, rY))
    vowel_mapper = {1791: "aah", 2548: "ee", 2314: "eh"}
    return X, y, vowel_mapper

##### Define an SVM and grid search over parameters in a cross validation

the predicted y is the cross-validated y

In [15]:
def run_svm(X,y):
    parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
    svr = svm.SVC()
    clf = grid_search.GridSearchCV(svr, parameters)
    y_pred = cross_val_predict(clf, X, y)
    return y_pred

##### Plotting functions

In [29]:
def plot_confusion_matrix(cm, mapper, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(mapper.keys()))
    foo = [vowel_mapper[q] for q in sorted(mapper.keys())]
    plt.xticks(tick_marks, foo, rotation=45)
    plt.yticks(tick_marks, foo)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

##### Show confusion matrix

In [24]:
def get_confusion_matrix(y,y_pred, mapper):
    cm = confusion_matrix(y, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plot_confusion_matrix(cm_normalized, mapper, title='Normalized confusion matrix')
    return cm_normalized

### Load data and run on every subject

In [25]:
subjects = sorted(glob("*.mat"))
print subjects

['s10_matfile.mat', 's12_matlfile.mat', 's2_matfile.mat', 's3_matfile.mat', 's4_matfiles.mat', 's5_matfiles.mat', 's6_matfile.mat', 's7_matfile.mat', 's8_matfile.mat', 's9_matfile.mat']


In [ ]:
result = {}
for s in subjects:
    data = sio.loadmat(s)
    X, y, mapper = get_variables(data)
    y_pred = run_svm(X,y)
    CM = get_confusion_matrix(y, y_pred, mapper)
    subname = s.split("_")[0]
    result[subname] = CM